# 02. Agentic Workflow: An In-Depth Codebase Recreation

Welcome to this detailed, educational walkthrough of a complete agentic workflow. This notebook is not just a demonstration but a faithful, step-by-step reconstruction of the core `multi-agent` inference engine. We will explore the exact, full-length prompts and the complete logic for each tool and component.

## What is an Multi-Agent System?

An agentic workflow, or a multi-agent system, is a sophisticated framework where a problem is solved by a series of specialized components (or "agents") that collaborate. Instead of relying on a single, monolithic LLM call to solve a complex query, this approach breaks the problem down into manageable stages:

1.  **Planning:** An agent analyzes the initial query and formulates a high-level plan or decides on the immediate next best action.
2.  **Tool Use:** The agent selects and uses specialized tools (like a code interpreter, a web search, or a database query tool) to gather information or perform actions.
3.  **Execution:** A dedicated component generates the precise command to run the selected tool.
4.  **Observation & Reflection:** The agent observes the result of the tool's execution and reflects on whether the goal has been achieved or if more steps are needed.
5.  **Iteration:** The process repeats in a loop until a verifier agent determines that the query has been fully answered.
6.  **Synthesis:** Finally, an agent synthesizes all the information gathered throughout the process into a comprehensive final answer.

This notebook will build each of these components from the ground up, providing a clear and powerful learning experience.

---

## Prerequisite: Start the vLLM Server

This entire workflow is powered by a Large Language Model. To run this notebook, you must first have a vLLM server running. vLLM is a high-throughput serving engine for LLMs that provides an OpenAI-compatible API endpoint.

Open a separate terminal and run the following command. You can replace the model with any other compatible model, but ensure the `MODEL_NAME` variable in the configuration cell below matches.

```bash
# Note: The model name in the command must match the MODEL_NAME variable in the notebook.
# Start the vLLM server with the same model name as MODEL_NAME below
# This can be a fine-tuned model or a base model, but it must match the name used in the notebook configuration.
vllm serve qwen-1.5-finetuned-grpo \ 
    --api-key a-secret-key \        # API key for OpenAI-compatible auth
    --port 8000 \                   # Port for the local server
    --max-model-len 8192            # Maximum context length
```

## Section 1: Setup, Dependencies, and Configuration

We begin by setting up our environment. This involves installing the required Python libraries, importing necessary modules, and configuring the connection to our vLLM server and other external services.

In [1]:
# This cell lists the required libraries for the project. 
# Uncomment the line below and run it if you haven't installed these packages yet.
# openai: The client library for interacting with OpenAI-compatible APIs (like our vLLM server).
# pydantic: For data validation and creating structured data models, crucial for reliable agent outputs.
# tenacity: A library for adding retry logic to functions, making our API calls more robust.
# beautifulsoup4 & requests: For fetching and parsing HTML content from websites (used in the Web_Search_Tool).
# wikipedia: A convenient wrapper for the Wikipedia API.
# google-genai: The client library for Google's Generative AI services (used in the Google_Search_Tool).
# numpy: For numerical operations, specifically for calculating vector similarity in our RAG tool.
# json_repair: A helpful utility to fix malformed JSON strings, which LLMs sometimes produce.

# !pip install -q openai==1.75.0 pydantic==2.5.3 tenacity==9.0.0 beautifulsoup4==4.12.3 requests==2.31.0 wikipedia==1.4.0 google-genai==0.8.5 numpy==1.26.4 json_repair

### 1.1 Imports and Global Configuration

Here, we import all necessary modules and define the core configuration variables. This includes the vLLM server URL, the model name, and placeholders for API keys required by our tools. 

**Crucially, you must replace the placeholder API keys with your own for the search tools to function correctly.**

In [1]:
# Standard library imports
import os
import json
import re
import sys
import inspect
import threading
from io import StringIO
from typing import Any, Dict, List, Union, Optional, Tuple
from abc import ABC, abstractmethod
from contextlib import contextmanager

# Pydantic and API Libraries
from pydantic import BaseModel
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_random_exponential
import requests
from bs4 import BeautifulSoup
import wikipedia
from google import genai
from google.genai import types
import numpy as np
import json_repair # For fixing malformed JSON from the LLM

In [2]:
# --- Core Configuration ---
# The base URL where your vLLM server is running.
VLLM_BASE_URL = "http://localhost:8000"
# The API key for your vLLM server (can be a dummy key if not required by your setup).
VLLM_API_KEY = "a-secret-key"
# The exact name of the model being served by vLLM or your fine-tuned model. This should match the model name configured in your vLLM server.
MODEL_NAME = "qwen-1.5-finetuned-grpo"

# --- Environment Variables for Tools ---
# IMPORTANT: You must provide your own API keys for the search tools to function.
# If you leave these as placeholders, the corresponding tools will operate in a 'mock' mode.
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY_HERE' # Needed for embeddings in Web_Search_Tool
os.environ['GOOGLE_API_KEY'] = 'YOUR_GOOGLE_API_KEY_HERE' # Needed for Google_Search_Tool

print(f"Notebook configured for model '{MODEL_NAME}' at {VLLM_BASE_URL}")

Notebook configured for model 'qwen-1.5-finetuned-grpo' at http://localhost:8000


## Section 2: The Core LLM Engine

To interact with our LLM, we'll create a wrapper class. This is a crucial design pattern that provides several benefits:
- **Abstraction:** It hides the specific details of the API calls, allowing us to easily swap out the backend (e.g., from vLLM to another provider) without changing the rest of our code.
- **Robustness:** We can build in features like automatic retries for failed API calls.
- **Feature Enhancement:** We can add custom logic, such as forcing the LLM to produce structured JSON output.

### 2.1 Abstract Base Class for the Engine

We start with an Abstract Base Class (ABC) to define a standard interface that all our LLM engine wrappers must follow. This ensures consistency and interchangeability. Any class that inherits from `EngineLM` *must* implement the `generate` method.

In [3]:
class EngineLM(ABC):
    """An abstract base class for a language model engine."""
    
    def __call__(self, *args, **kwargs):
        """Allows the class instance to be called like a function, making the syntax cleaner."""
        return self.generate(*args, **kwargs)

    @abstractmethod
    def generate(self, prompt, system_prompt=None, **kwargs):
        """The core method that must be implemented by any subclass. It takes a prompt and generates a response."""
        pass

### 2.2 The vLLM Chat Engine Implementation

This is our concrete implementation for interacting with the vLLM server. It handles prompt formatting, API calls, and, most importantly, the logic for parsing structured JSON output. This class is the heart of our agent's "brain".

In [4]:
class ChatVLLM(EngineLM):
    """A language model engine that connects to a vLLM server with an OpenAI-compatible API."""
    
    def __init__(self, model_string, base_url, api_key, temperature=0.0):
        """Initializes the engine with connection details and the OpenAI client."""
        self.model_string = model_string
        self.base_url = base_url
        self.api_key = api_key
        self.temperature = temperature
        # The OpenAI client is configured to point to our local vLLM server.
        self.client = OpenAI(base_url=self.base_url, api_key=self.api_key)
        self.default_system_prompt = "You are a helpful, creative, and smart assistant."

    @retry(wait=wait_random_exponential(min=1, max=5), stop=stop_after_attempt(3))
    def generate(self, content: Union[str, List[Union[str, bytes]]], system_prompt=None, response_format=None, **kwargs):
        """Generates a response from the LLM, with robust JSON parsing and retry logic."""
        # Use the provided system prompt or fall back to the default.
        sys_prompt_arg = system_prompt if system_prompt else self.default_system_prompt
        user_content = content

        # Format the request in the standard chat completions message format.
        messages = [
            {"role": "system", "content": sys_prompt_arg},
            {"role": "user", "content": user_content}
        ]

        # Prepare the parameters for the API request.
        request_params = {
            "model": self.model_string,
            "messages": messages,
            "temperature": self.temperature,
            "max_tokens": kwargs.get("max_tokens", 4096),
        }

        # This block is key for structured output. If a Pydantic model is provided,
        # we serialize its JSON schema and append it to the prompt, instructing the LLM
        # to format its response accordingly. This is a form of "prompt engineering".
        if response_format and issubclass(response_format, BaseModel):
            json_schema = response_format.model_json_schema()
            schema_instruction = (f"\n\nPlease format your entire response as a single JSON object that strictly adheres to the following Pydantic schema.\n"+
                                f"Do not include any other text, explanations, or markdown formatting outside of the JSON object.\n\n"+
                                f"Schema:\n```json\n{json.dumps(json_schema, indent=2)}\n```")
            # Append the instruction to the last user message.
            if isinstance(request_params['messages'][-1]['content'], str):
                 request_params['messages'][-1]['content'] += schema_instruction

        print(f"\n{'─'*50}\n[LLM_INPUT] Sending request to model: {self.model_string}\n{'─'*50}")
        
        try:
            # Execute the API call to the vLLM server.
            response = self.client.chat.completions.create(**request_params)
            response_text = response.choices[0].message.content
        except Exception as e:
            print(f"[LLM ERROR] API Call Failed: {e}")
            raise e

        print(f"\n{'─'*50}\n[LLM_OUTPUT] Raw response from model:\n{response_text}\n{'─'*50}")

        # If we requested a structured format, we now parse the response.
        if response_format and issubclass(response_format, BaseModel):
            # First, try to extract the JSON from within markdown code blocks (e.g., ```json ... ```).
            match = re.search(r"```json\s*(.*?)\s*```", response_text, re.DOTALL)
            json_str = match.group(1) if match else response_text
            
            try:
                # Attempt to parse the JSON strictly using the Pydantic model. This enforces the schema.
                parsed_obj = response_format.model_validate_json(json_str.strip())
                print("[LLM Engine] Successfully parsed structured output (Strict).")
                return parsed_obj
            except Exception as e:
                # If strict parsing fails (e.g., due to trailing commas, missing quotes), we attempt to repair it.
                print(f"[LLM Engine info] Strict parsing failed ({str(e)[:100]}...). Attempting repair...")
                try:
                    # The `json_repair` library can fix many common LLM-generated JSON errors.
                    decoded_obj = json_repair.loads(json_str)
                    # After repairing, we validate the repaired object against the Pydantic model again.
                    parsed_obj = response_format.model_validate(decoded_obj)
                    print("[LLM Engine] Successfully parsed structured output (Repaired).")
                    return parsed_obj
                except Exception as e2:
                    # If even the repair fails, we log a critical warning. Returning the raw text allows
                    # the agent to potentially see the error, but it might crash the next step.
                    print(f"[LLM Engine CRITICAL WARNING] Failed to parse output even with repair: {e2}")
                    return response_text

        # If no structured format was requested, return the raw text response.
        return response_text

### 2.3 Engine Factory and Connection Test

A factory function is a clean way to instantiate our engine. We then immediately test the connection to the vLLM server to ensure everything is configured correctly before proceeding.

In [5]:
def create_llm_engine(model_string: str, **kwargs) -> ChatVLLM:
    """Factory function to create an instance of our vLLM chat engine."""
    return ChatVLLM(model_string=model_string, base_url=VLLM_BASE_URL, api_key=VLLM_API_KEY, temperature=kwargs.get('temperature', 0.0))

# --- Test Connection ---
try:
    print("--- Testing vLLM engine connection ---")
    # Create an engine instance.
    test_engine = create_llm_engine(MODEL_NAME)
    # Send a simple message to see if we get a response.
    test_response = test_engine.generate("Ping")
    print(f"\n✅ Connection successful!")
except Exception as e:
    print(f"\n❌ CONNECTION FAILED: {e}")

--- Testing vLLM engine connection ---

──────────────────────────────────────────────────
[LLM_INPUT] Sending request to model: qwen-1.5-finetuned-grpo
──────────────────────────────────────────────────

──────────────────────────────────────────────────
[LLM_OUTPUT] Raw response from model:
Pong
──────────────────────────────────────────────────

✅ Connection successful!


## Section 3: Defining the Agent's Data Structures (Pydantic Models)

A robust agentic system relies on predictable, structured data flowing between its components. We use Pydantic models to define the schemas for these data structures. This provides:

- **Type Safety:** Ensures data conforms to the expected types (e.g., a field is a string, not a list).
- **Validation:** Automatically checks if the data from the LLM is valid and complete.
- **Self-Documentation:** The models themselves serve as clear documentation for what each component expects as input and produces as output.
- **Reliable Communication:** They form the contract between the different LLM-powered "roles" in our system (Planner, Verifier, etc.).

### 3.1 Query Analysis Model

This model defines the structure of the initial analysis of the user's query. It's the output of the first step, where the Planner agent breaks down the problem.

In [6]:
class QueryAnalysis(BaseModel):
    """Represents the initial breakdown and analysis of the user's query."""
    concise_summary: str
    required_skills: str
    relevant_tools: str
    additional_considerations: str

### 3.2 Next Step Model

At each iteration of the main loop, the Planner agent decides on the next action. This model captures that decision, specifying which tool to use and for what purpose.

In [7]:
class NextStep(BaseModel):
    """Defines the plan for the next action to be taken in the agent's loop."""
    justification: str
    context: str
    sub_goal: str
    tool_name: str

### 3.3 Tool Command Model

Once a tool and sub-goal are chosen, the Executor agent's job is to generate the exact code to run that tool. This model structures the output of the Executor.

In [8]:
class ToolCommand(BaseModel):
    """Represents the generated command for a specific tool, ready for execution."""
    analysis: str
    explanation: str
    command: str

### 3.4 Memory Verification Model

After an action is executed, the Verifier agent reflects on the current state. This model captures its conclusion: should the agent stop, or does it need to continue with more steps?

In [9]:
class MemoryVerification(BaseModel):
    """Represents the verifier's analysis of whether the task is complete."""
    analysis: str
    stop_signal: bool

### 3.5 Relevant Queries Model (for Wikipedia Tool)

This is a specialized model used internally by the `Wikipedia_Search_Tool`. When searching Wikipedia, it gets a list of possible page titles and uses an LLM call with this response format to select the most relevant pages to investigate further.

In [10]:
class Select_Relevant_Queries(BaseModel):
    """A specialized model for the Wikipedia tool to select relevant search results."""
    matched_queries: list[str]
    matched_query_ids: list[int]

## Section 4: The Agent's Memory

An agent needs a way to remember what it has done. The `Memory` class serves as a simple logbook, recording each action taken, the tool used, the command executed, and the result obtained. This history is fed back into the Planner and Verifier agents in subsequent steps, providing them with the context needed to make informed decisions.

In [11]:
class Memory:
    """A simple class to store the history of actions taken by the agent."""
    def __init__(self):
        """Initializes an empty dictionary to store actions."""
        self.actions: Dict[str, Dict[str, Any]] = {}
        
    def add_action(self, step_count: int, tool_name: str, sub_goal: str, command: str, result: Any) -> None:
        """Adds a new action to the memory log."""
        self.actions[f"Action Step {step_count}"] = {
            'tool_name': tool_name, 
            'sub_goal': sub_goal, 
            'command': command, 
            'result': result
        }
        
    def get_actions(self) -> Dict[str, Dict[str, Any]]:
        """Retrieves the entire history of actions."""
        return self.actions

### 4.1 Memory Serialization Helper

Tool outputs can be complex objects or very long strings. Before storing a result in memory (and feeding it back into an LLM's limited context window), it's essential to serialize it into a clean, truncated JSON format. This function handles that conversion recursively.

In [12]:
def make_json_serializable_truncated(obj, max_length: int = 2000):
    """Recursively converts an object into a JSON-serializable and truncated format."""
    # Handle basic, JSON-native types.
    if isinstance(obj, (int, float, bool, type(None))): return obj
    # Truncate long strings.
    elif isinstance(obj, str): return obj if len(obj) <= max_length else obj[:max_length - 3] + "..."
    # Recursively process dictionaries.
    elif isinstance(obj, dict): return {str(k): make_json_serializable_truncated(v, max_length) for k, v in obj.items()}
    # Recursively process lists.
    elif isinstance(obj, list): return [make_json_serializable_truncated(element, max_length) for element in obj]
    # For all other types, convert to a string representation and truncate.
    else:
        result = repr(obj)
        return result if len(result) <= max_length else result[:max_length - 3] + "..."

## Section 5: Building the Tools

Tools are what give an agent its power. They are functions that allow the agent to interact with the outside world, perform calculations, or access information beyond its own knowledge. By giving the agent access to tools, we ground its reasoning and enable it to solve much more complex problems.

We start by defining a `BaseTool` abstract class to ensure all our tools have a consistent structure and expose metadata about their capabilities. This metadata is crucial for the Planner agent to decide which tool is appropriate for a given task.

In [13]:
class BaseTool(ABC):
    """An abstract base class that defines the standard interface for all tools."""
    # A flag to indicate if the tool requires an LLM engine to function.
    require_llm_engine = False
    
    def __init__(self, tool_name=None, tool_description=None, tool_version=None, input_types=None, output_type=None, demo_commands=None, user_metadata=None, model_string=None):
        """Initializes the tool with its descriptive metadata."""
        self.tool_name, self.tool_description, self.tool_version, self.input_types, self.output_type, self.demo_commands, self.user_metadata, self.model_string = tool_name, tool_description, tool_version, input_types, output_type, demo_commands, user_metadata, model_string
    
    def get_metadata(self) -> dict:
        """Returns all the tool's metadata in a dictionary. This is fed to the Planner agent."""
        return {"tool_name": self.tool_name, "tool_description": self.tool_description, "tool_version": self.tool_version, "input_types": self.input_types, "output_type": self.output_type, "demo_commands": self.demo_commands, "user_metadata": self.user_metadata, "require_llm_engine": self.require_llm_engine}
    
    def set_custom_output_dir(self, output_dir):
        """A placeholder method for tools that might need to save files."""
        pass
        
    @abstractmethod
    def execute(self, *args, **kwargs):
        """The core method where the tool's logic is implemented. Must be overridden by subclasses."""
        raise NotImplementedError

### 5.1 Generalist Solution Generator Tool

This is the simplest tool. It's essentially a direct, unconstrained call to the LLM. It acts as a fallback or a general-purpose reasoning tool when no specialized tool is a good fit.

In [14]:
class Base_Generator_Tool(BaseTool):
    """A general-purpose tool that uses an LLM to answer a query directly."""
    # This tool's primary function is to call an LLM, so it requires an engine.
    require_llm_engine = True
    
    def __init__(self, model_string="gpt-4o-mini"):
        """Initializes the tool's metadata and its own LLM engine."""
        super().__init__(
            tool_name="Generalist_Solution_Generator_Tool", 
            tool_description="A generalized tool that takes query from the user, and answers the question step by step to the best of its ability.", 
            tool_version="1.0.0", 
            input_types={"query": "str"}, 
            output_type="str", 
            user_metadata={
                "limitation": "The Generalist_Solution_Generator_Tool may provide hallucinated or incorrect responses.", 
                "best_practice": "Use for general queries. Verify important information from its responses."
            }
        )
        self.llm_engine = create_llm_engine(model_string, temperature=0.0)
    
    def execute(self, query, **kwargs):
        """Executes the tool by passing the query directly to its LLM engine."""
        return self.llm_engine.generate([query])

### 5.2 Python Coder Tool

This tool allows the agent to write and execute Python code. This is extremely powerful for tasks involving calculations, data manipulation, or logical operations. The implementation includes important safety features:

- **Sandboxing:** The code is executed using `exec()` in a controlled, empty scope to prevent it from accessing or modifying the main program's state.
- **Timeout:** A timer prevents the code from running indefinitely, which is crucial for handling infinite loops or long-running computations.
- **Output Capturing:** It captures any `print` statements from the executed code, which is often how the result of a calculation is exposed.

In [15]:
class TimeoutException(Exception):
    """Custom exception to be raised when an operation times out."""
    pass

@contextmanager
def timeout(seconds):
    """A context manager to enforce a timeout on a block of code."""
    # Define a function that will be called by the timer to raise the exception.
    def raise_timeout(signum, frame):
        raise TimeoutException("Code execution timed out")
    
    # Use a threading.Timer to run the raise_timeout function after a delay.
    timer = threading.Timer(seconds, lambda: raise_timeout(None, None))
    timer.start()
    try:
        # The 'yield' passes control back to the 'with' block.
        yield
    finally:
        # This code always runs, whether the 'with' block finished or an exception occurred.
        # It's crucial to cancel the timer to prevent the timeout from firing later.
        timer.cancel()

class Python_Coder_Tool(BaseTool):
    """A tool to generate and execute Python code in a sandboxed environment."""
    require_llm_engine = True
    
    def __init__(self, model_string="gpt-4o"):
        super().__init__(
            tool_name="Python_Code_Generator_Tool", 
            tool_description="A tool that generates and executes simple Python code snippets for basic arithmetical calculations and math-related problems.", 
            tool_version="1.0.0", 
            input_types={"query": "str"}, 
            output_type="dict", 
            user_metadata={
                "limitations": "Restricted to basic Python arithmetic and built-in math functions. Cannot use external libraries, file I/O, or network requests. Execution times out after 10 seconds.", 
                "best_practices": "Provide clear queries with all necessary numerical inputs. Good for math and logic problems."
            }
        )
        self.llm_engine = create_llm_engine(model_string, temperature=0.0)

    def execute(self, query, **kwargs):
        # 1. Prompt an LLM to generate the Python code.
        task_description = "Given a query, generate a Python code snippet that performs the specified operation. Ensure to print the final result. The final output should be presented in the following format:\n\n```python\n<code snippet>\n```"
        full_prompt = f"Task:\n{task_description}\n\nQuery:\n{query}"
        response = self.llm_engine.generate(full_prompt)
        
        # 2. Extract the code from the LLM's response.
        match = re.search(r"```python\s*(.*?)\s*```", response, re.DOTALL)
        if not match: return {"error": "No Python code block found in the response", "raw_response": response}
        code_snippet = match.group(1).strip()
        
        # 3. Execute the code in a safe, controlled environment.
        output_capture = StringIO() # Create an in-memory text buffer to capture print statements.
        old_stdout, old_stderr = sys.stdout, sys.stderr # Store the original stdout/stderr
        local_vars = {} # A dictionary to hold variables created by the executed code.
        try:
            # Redirect stdout and stderr to our in-memory buffer.
            sys.stdout = sys.stderr = output_capture
            with timeout(10): # Enforce a 10-second timeout.
                # `exec` runs the code. We provide empty global and a local dict for the scope.
                exec(code_snippet, {}, local_vars)
            printed_output = output_capture.getvalue().strip()
            # Return the captured output and any variables created by the code.
            return {"printed_output": printed_output, "variables": {k: repr(v) for k, v in local_vars.items() if not k.startswith('__')}}
        except TimeoutException as e: return {"error": str(e), "code": code_snippet}
        except Exception as e: return {"error": str(e), "code": code_snippet, "captured_output": output_capture.getvalue().strip()}
        finally: 
            # CRITICAL: Always restore the original stdout and stderr.
            sys.stdout, sys.stderr = old_stdout, old_stderr

### 5.3 Google Search Tool

This tool gives the agent the ability to search the web for real-time information. It uses the Google Gemini API's built-in grounding feature, which is a simple and effective way to perform grounded generation based on search results. If no `GOOGLE_API_KEY` is provided, it will run in a mock mode, returning a placeholder string.

In [ ]:
class Google_Search_Tool(BaseTool):
    """A tool for performing web searches using Google's Gemini API with grounding."""
    def __init__(self, model_string="gemini-1.5-flash"):
        super().__init__(
            tool_name="Ground_Google_Search_Tool", 
            tool_description="A web search tool powered by Google's Gemini AI that provides real-time information.", 
            tool_version="1.0.0", 
            input_types={"query": "str"}, 
            output_type="str", 
            user_metadata={"limitations": "Only suitable for general information search.", "best_practices": "Choose for question-type queries."}
        )
        self.search_model = model_string
        
        # Check for a valid API key. If not present, set client to None to enable mock mode.
        if not os.getenv("GOOGLE_API_KEY") or 'YOUR_GOOGLE_API_KEY' in os.getenv("GOOGLE_API_KEY"):
            print("WARNING: Google_Search_Tool is in mock mode. Provide a GOOGLE_API_KEY to enable.")
            self.client = None
        else:
            # We'll use the recommended `genai.GenerativeModel` for modern usage, but the logic is similar.
            genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
            self.client = genai.GenerativeModel(self.search_model)

    def execute(self, query: str, add_citations: bool = True, **kwargs):
        # If in mock mode, return a placeholder response.
        if not self.client: 
            return f"[Mock Response] Search results for: '{query}'"
        
        try:
            # Configure the Gemini API to use its internal Google Search tool for grounding.
            grounding_tool = genai.Tool(
                google_search=genai.GoogleSearch()
            )
            # Generate content with the search tool enabled.
            response = self.client.generate_content(
                query,
                tools=[grounding_tool],
            )
            return response.text
        except Exception as e:
            return f"Error during Google Search: {e}"

### 5.4 Web RAG Search Tool

This tool implements a complete Retrieval-Augmented Generation (RAG) pipeline to answer a query based on the content of a specific URL. It's more targeted than a general web search. The process is:

1.  **Fetch & Parse:** Downloads the HTML from the URL and extracts all the clean text.
2.  **Chunk:** Splits the long text into smaller, manageable chunks.
3.  **Embed:** Uses an embedding model (from OpenAI in this case) to convert the user's query and each text chunk into numerical vectors.
4.  **Retrieve:** Calculates the cosine similarity between the query vector and all chunk vectors to find the chunks that are most semantically relevant to the query.
5.  **Synthesize:** Passes the original query and the content of the most relevant chunks to an LLM, asking it to synthesize a final answer based *only* on the provided context.

In [17]:
class Web_Search_Tool(BaseTool):
    """Answers questions by retrieving info from a website using a RAG pipeline."""
    require_llm_engine = True
    
    def __init__(self, model_string="gpt-4o-mini"):
        super().__init__(tool_name="Web_RAG_Search_Tool", tool_description="Answers questions by retrieving info from a website using RAG.", tool_version="1.0.0", input_types={"query": "str", "url": "str"}, output_type="str", user_metadata={"limitation": "May not work with JS-heavy sites or those requiring authentication.", "best_practice": "Use specific, targeted queries on text-rich websites."})
        self.llm_engine = create_llm_engine(model_string, temperature=0.0)
        # This tool requires an OpenAI key for its embedding model.
        if not os.getenv("OPENAI_API_KEY") or 'YOUR_OPENAI_API_KEY' in os.getenv("OPENAI_API_KEY"):
            print("WARNING: Web_Search_Tool is in mock mode. Provide an OPENAI_API_KEY to enable embeddings."); self.embedding_client = None
        else: self.embedding_client = OpenAI()
    
    def execute(self, query, url, **kwargs):
        if not self.embedding_client: return f"[Mock Response] RAG summary for query '{query}' on URL '{url}'"
        try:
            # 1. Fetch & Parse: Use requests and BeautifulSoup to get text from the URL.
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
            content = BeautifulSoup(requests.get(url, headers=headers, timeout=10).content, 'html.parser').get_text(separator='\n', strip=True)
            
            # 2. Chunk: Split the text into overlapping chunks of 200 words.
            words = content.split(); chunks = [" ".join(words[i:i+200]) for i in range(0, len(words), 180)]
            
            # 3. Embed: Get embeddings for the query and all chunks in a single API call.
            embeddings = self.embedding_client.embeddings.create(input=[query] + chunks, model="text-embedding-3-small").data
            query_embedding, chunk_embeddings = np.array(embeddings[0].embedding), np.array([e.embedding for e in embeddings[1:]])
            
            # 4. Retrieve: Calculate cosine similarity and get the top 10 most relevant chunks.
            similarities = [np.dot(query_embedding, ce) / (np.linalg.norm(query_embedding) * np.linalg.norm(ce)) for ce in chunk_embeddings]
            top_chunks = [chunks[i] for i in np.argsort(similarities)[-10:][::-1]]
            reference_info = "\n".join([f"[{i+1}] {chunk}" for i, chunk in enumerate(top_chunks)])
            
            # 5. Synthesize: Prompt a separate LLM to generate an answer based on the retrieved chunks.
            summary_prompt = f"You are an expert AI assistant. Your task is to provide a clear, concise, and accurate answer to the user's query based **exclusively** on the provided reference information.\n\n## Step-by-Step Instructions\n1.  **Analyze the Query:** First, fully understand the user's query and identify the specific information being asked for.\n2.  **Scan for Relevance:** Read through each numbered chunk in the reference information. Identify all chunks that contain information directly relevant to answering the query.\n3.  **Extract Key Facts & Synthesize:** From the relevant chunks, extract only the key facts and figures needed. Synthesize these extracted facts into a comprehensive, single-paragraph answer.\n\n---\n## Your Turn\n\n### User Query\n{query}\n\n### Reference Information\n{reference_info}\n\n### Output\n"
            return self.llm_engine.generate(summary_prompt)
        except Exception as e: return f"Error in Web_Search_Tool: {e}"

### 5.5 Wikipedia RAG Search Tool

This is a *composite tool* or a *meta-tool*. It orchestrates other components to perform its task. Its process is:

1.  **Search:** Use the `wikipedia` library to get a list of potential page titles related to the user's query.
2.  **Select:** Use an LLM call (with the `Select_Relevant_Queries` Pydantic model) to intelligently filter this list down to the most promising candidates.
3.  **Process:** For each selected page, it calls the `Web_Search_Tool` (which it holds an instance of) to perform a full RAG pipeline on that specific Wikipedia page.
4.  **Aggregate:** It returns a structured dictionary containing the retrieved information from all the relevant pages it processed.

In [18]:
class Wikipedia_Search_Tool(BaseTool):
    """A composite tool that searches Wikipedia, selects relevant pages, and applies RAG."""
    require_llm_engine = True
    
    def __init__(self, model_string="gpt-4o-mini"):
        super().__init__(tool_name="Wikipedia_RAG_Search_Tool", tool_description="Searches Wikipedia and uses RAG to get grounded information from pages.", tool_version="1.0.0", input_types={"query": "str"}, output_type="dict", user_metadata={"limitation": "Wikipedia only. Accuracy depends on Wikipedia content. Filtering of pages depends on LLM performance.", "best_practice": "Use specific, targeted queries. Trust the 'relevant_pages' results."})
        self.llm_engine = create_llm_engine(model_string, temperature=0.0)
        # This tool internally uses another tool.
        self.web_rag_tool = Web_Search_Tool(model_string=model_string)
    
    def execute(self, query, **kwargs):
        try:
            # 1. Search: Get up to 10 potential page titles from the Wikipedia API.
            search_results = wikipedia.search(query, results=10)
            if not search_results: return {"error": f"No results found for '{query}'"}
            
            # 2. Select: Prompt an LLM to choose the most relevant titles from the search results.
            query_candidates_str = "\n".join([f"{i}. {query}" for i, query in enumerate(search_results)])
            prompt = f"""You are an expert AI assistant. Your task is to identify and select the most relevant queries from a list of Wikipedia search results that are most likely to address the user’s original question.\n\n## Input\n\nOriginal Query: `{query}`\nQuery Candidates from Wikipedia Search: `{query_candidates_str}`\n\n## Instructions\n1. Carefully read the original query and the list of query candidates.\n2. Select the query candidates that are most relevant to the original query.\n3. Return up to 3 most relevant queries."""

            selection = self.llm_engine.generate(prompt, response_format=Select_Relevant_Queries)
            # Fallback logic in case the LLM fails to produce a valid structured response.
            if not isinstance(selection, Select_Relevant_Queries):
                print("Warning: Failed to parse relevant queries, using first result as fallback.")
                selection = Select_Relevant_Queries(matched_queries=[search_results[0]], matched_query_ids=[0])

            # 3. Process & Aggregate: Loop through the selected titles.
            relevant_pages = []
            for title in selection.matched_queries:
                try:
                    # Get the full page object from the Wikipedia API.
                    page = wikipedia.page(title, auto_suggest=False)
                    # Use the Web_RAG_Tool to process the content of the page's URL.
                    info = self.web_rag_tool.execute(query=query, url=page.url)
                    relevant_pages.append({"title": title, "url": page.url, "retrieved_information": info})
                except Exception as page_e: 
                    # Handle cases where a page might be a disambiguation page or cause an error.
                    relevant_pages.append({"title": title, "url": f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}", "error": str(page_e)})
            # Return a structured dictionary of the findings.
            return {"query": query, "relevant_pages": relevant_pages}
        except Exception as e: return {"error": f"Wikipedia search failed: {e}"}

## Section 6: The Agent's Orchestration Engine

Now we bring all the pieces together. The orchestration engine is responsible for managing the agent's lifecycle, from initializing the tools to running the main problem-solving loop.

### 6.1 The Tool Initializer

This class handles the setup of the agent's toolbox. It takes a list of desired tools, instantiates their respective classes, and stores both the tool instances (for execution) and their metadata (for the Planner agent). This separation of concerns keeps the main `Solver` class cleaner.

In [19]:
class Initializer:
    """Handles the loading and configuration of all tools for the agent."""
    def __init__(self, enabled_tools: List[str], tool_engine: List[str], model_string: str):
        self.toolbox_metadata, self.tool_instances_cache, self.available_tools = {}, {}, []
        print("\n==> Initializing agent..."); self._set_up_tools(enabled_tools, tool_engine, model_string)
        
    def _set_up_tools(self, enabled_tools: List[str], tool_engine: List[str], model_string: str):
        print(f"Enabled tools: {enabled_tools} with engines: {tool_engine}")
        # A registry of all available tool classes.
        all_tool_classes = {
            "Base_Generator_Tool": Base_Generator_Tool, 
            "Python_Coder_Tool": Python_Coder_Tool, 
            "Google_Search_Tool": Google_Search_Tool, 
            "Wikipedia_RAG_Search_Tool": Wikipedia_Search_Tool
        }
        # Loop through the list of tools to enable.
        for i, tool_class_name in enumerate(enabled_tools):
            if tool_class_name in all_tool_classes:
                tool_class = all_tool_classes[tool_class_name]
                # Determine which LLM engine this tool instance should use.
                engine = tool_engine[i] if i < len(tool_engine) else model_string
                print(f"  -> Loading '{tool_class_name}' with engine '{engine}'...")
                # Create an instance of the tool class.
                instance = tool_class() if engine == "Default" else tool_class(model_string=engine)
                ext_name = instance.tool_name
                # Store the instance for execution and its metadata for planning.
                self.tool_instances_cache[ext_name] = instance
                self.toolbox_metadata[ext_name] = instance.get_metadata()
                self.available_tools.append(ext_name)
                print(f"     ✓ Loaded and cached as '{ext_name}'")
        print(f"\n✅ Tool setup complete. Final available tools: {self.available_tools}")

### 6.2 The Solver: The Main Agent Loop

The `Solver` class is the master orchestrator. It embodies the entire agentic workflow. It initializes all the necessary components and contains the primary `solve` method which executes the multi-step reasoning loop.

The `__init__` method sets up the different "roles" of the agent by creating separate LLM engine instances for each task. While they all point to the same model in this notebook, in a more advanced system they could be different models specialized for planning, verification, or code generation.

In [20]:
class Solver:
    """The main class that orchestrates the entire agentic problem-solving workflow."""
    def __init__(self, planner_main_engine, planner_fixed_engine, verifier_engine, executor_engine, enabled_tools, tool_engine, max_steps=5):
        """Initializes all components of the agent: LLM engines, tools, and memory."""
        self.max_steps = max_steps
        print("\n==> Initializing LLM engines for different roles...")
        # Initialize an LLM engine for each distinct role in the workflow.
        self.llm_planner_main = create_llm_engine(planner_main_engine); print(f"  - Planner (Main):      {planner_main_engine}")
        self.llm_planner_fixed = create_llm_engine(planner_fixed_engine); print(f"  - Planner (Fixed/Aux): {planner_fixed_engine}")
        self.llm_verifier = create_llm_engine(verifier_engine); print(f"  - Verifier:            {verifier_engine}")
        self.llm_executor = create_llm_engine(executor_engine); print(f"  - Executor:            {executor_engine}")
        
        # Use the Initializer class to set up the toolbox.
        initializer = Initializer(enabled_tools, tool_engine, planner_main_engine)
        self.tool_instances_cache = initializer.tool_instances_cache
        self.toolbox_metadata = initializer.toolbox_metadata
        self.available_tools = initializer.available_tools
        
        # Initialize the agent's memory.
        self.memory = Memory()


### 6.3 The `solve` Method: Step-by-Step Execution

This is the core logic. The `solve` method takes a user query and executes the agentic loop until the problem is solved or the maximum number of steps is reached. Let's break down each phase of the loop.

#### Step 0: Initial Query Analysis
Before the loop begins, the agent performs a one-time analysis of the query to create a high-level understanding of the user's intent, the skills required, and the tools that might be relevant. This analysis provides context for all subsequent steps.

In [21]:
def solve_step_0_analyze(self, question: str) -> QueryAnalysis:
    """Performs the initial analysis of the user's query."""
    print(f"\n{'='*80}\n==> 🔍 Received Query: {question}\n{'='*80}")
    # Clear memory from any previous runs.
    self.memory.actions.clear()

    print("\n==> 🔍 Step 0: Query Analysis (using planner_fixed_engine)")
    # This is the prompt that instructs the LLM on how to analyze the query.
    # It's given the query, the list of available tools, and the detailed tool metadata.
    prompt_analyze_query = f"""Task: Analyze the given query to determine necessary skills and tools.

Inputs:
- Query: {question}
- Available tools: {json.dumps(self.available_tools)}
- Metadata for tools: {json.dumps(self.toolbox_metadata, indent=2)}

Instructions:
1. Identify the main objectives in the query.
2. List the necessary skills and tools.
3. For each skill and tool, explain how it helps address the query.
4. Note any additional considerations.

Format your response with a summary of the query, lists of skills and tools with explanations, and a section for additional considerations. Be brief and precise with insight."""
    # The LLM is asked to respond in the format of the QueryAnalysis Pydantic model.
    query_analysis = self.llm_planner_fixed.generate(prompt_analyze_query, response_format=QueryAnalysis)
    print(f"\n--- Analysis Result ---\n{json.dumps(query_analysis.model_dump(), indent=2)}")
    return query_analysis

# We will attach this method to the Solver class later.
Solver.solve_step_0_analyze = solve_step_0_analyze

#### Step {i}.1: Planning the Next Action
At the start of each loop iteration, the Planner agent decides what to do next. It considers the original query, the initial analysis, the history of all previous actions stored in memory, and the available tools. Its goal is to choose the single best tool and define a clear, specific sub-goal for it to achieve in this step.

In [22]:
def solve_step_1_plan(self, question: str, query_analysis: QueryAnalysis, step_count: int) -> NextStep:
    """Plans the next best action for the current step of the loop."""
    print(f"\n{'='*80}\n==> 🎯 Step {step_count}: Planning Next Action (using planner_main_engine)\n{'='*80}")
            
    # The prompt for the Planner. It receives all current context.
    prompt_next_step = f"""Task: Determine the optimal next step to address the query using available tools and previous steps.

Context:
- **Query:** {question}
- **Query Analysis:** {query_analysis}
- **Available Tools:** {json.dumps(self.available_tools)}
- **Toolbox Metadata:** {json.dumps(self.toolbox_metadata, indent=2)}
- **Previous Steps:** {json.dumps(self.memory.get_actions())}

Instructions:
1. Analyze the query, previous steps, and available tools.
2. Select the **single best tool** for the next step.
3. Formulate a specific, achievable **sub-goal** for that tool.
4. Provide all necessary **context** (data, file names, variables) for the tool to function.

Response Format:
1.  **Justification:** Explain your choice of tool and sub-goal.
2.  **Context:** Provide all necessary information for the tool.
3.  **Sub-Goal:** State the specific objective for the tool.
4.  **Tool Name:** State the exact name of the selected tool."""
    # The LLM must respond in the format of the NextStep Pydantic model.
    next_step = self.llm_planner_main.generate(prompt_next_step, response_format=NextStep)
    print(f"[Planner Justification]: {next_step.justification}\n[Selected Tool]: {next_step.tool_name}\n[Sub-Goal]: {next_step.sub_goal}")
    return next_step

Solver.solve_step_1_plan = solve_step_1_plan

#### Step {i}.2: Generating the Tool Command
Once a tool and sub-goal are chosen, the Executor agent takes over. Its job is to translate the high-level sub-goal into a precise, executable piece of Python code that calls the selected tool with the correct arguments.

In [23]:
def solve_step_2_generate_command(self, question: str, next_step: NextStep, step_count: int) -> str:
    """Generates the executable Python command for the chosen tool."""
    print(f"\n==> 📝 Step {step_count}: Generating Command for '{next_step.tool_name}' (using executor_engine)")
    # The prompt for the Executor. It gets the sub-goal, tool metadata, and relevant context.
    prompt_tool_command = f"""Task: Generate a precise command to execute the selected tool.

Context:
- **Query:** {question}
- **Sub-Goal:** {next_step.sub_goal}
- **Tool Name:** {next_step.tool_name}
- **Tool Metadata:** {self.toolbox_metadata.get(next_step.tool_name, {})}
- **Relevant Data:** {next_step.context}

Instructions:
1.  Analyze the tool's required parameters from its metadata.
2.  Construct valid Python code that addresses the sub-goal using the provided context and data.
3.  The command must include at least one call to `tool.execute()`.
4.  Each `tool.execute()` call must be assigned to a variable named **`execution`**.
5.  Please give the exact numbers and parameters should be used in the `tool.execute()` call.
"""
    # The LLM must respond in the format of the ToolCommand Pydantic model.
    tool_command_obj = self.llm_executor.generate(prompt_tool_command, response_format=ToolCommand)
    command_to_run = tool_command_obj.command.strip()
    print(f"[Generated Command]:\n```python\n{command_to_run}\n```")
    return command_to_run

Solver.solve_step_2_generate_command = solve_step_2_generate_command

#### Step {i}.3: Executing the Command
This is where the agent interacts with the world. The generated command is executed. The tool instance is made available in the local scope of the `exec` call, and the result is captured. Any errors during execution are caught and stored in the result, so the agent can see that its command failed and potentially correct it in the next step.

In [24]:
def solve_step_3_execute_command(self, next_step: NextStep, command_to_run: str, step_count: int):
    """Executes the generated command and stores the result in memory."""
    print(f"\n==> 🛠️ Step {step_count}: Executing Command for '{next_step.tool_name}'")
    tool_instance = self.tool_instances_cache.get(next_step.tool_name)
    local_context = {'tool': tool_instance}
    
    if not tool_instance:
        result = f"Error: Tool '{next_step.tool_name}' not found."
    else:
        try:
            # Execute the command. The result must be stored in a variable named 'execution'.
            exec(command_to_run, {}, local_context)
            result = local_context.get('execution', "Error: No 'execution' variable returned.")
        except Exception as e:
            result = f"Execution Error: {str(e)}"
    
    # Sanitize and truncate the result before adding it to memory.
    serializable_result = make_json_serializable_truncated(result)
    self.memory.add_action(step_count, next_step.tool_name, next_step.sub_goal, command_to_run, serializable_result)
    print(f"[Execution Result]:\n{json.dumps(serializable_result, indent=2)}")

#### Step {i}.4: Verifying and Reflecting
After executing an action, the agent must pause and reflect. The Verifier agent reviews everything—the original query, the initial analysis, and the full memory of actions and results—to determine if the query has been fully answered. Its output is a simple but critical boolean: `stop_signal`. If `True`, the loop terminates. If `False`, the agent proceeds to the next iteration.

In [25]:
def solve_step_4_verify(self, question: str, query_analysis: QueryAnalysis, step_count: int) -> bool:
    """Verifies if the task is complete, returning True to stop or False to continue."""
    print(f"\n==> 🤖 Step {step_count}: Verifying Context (using verifier_engine)")
    # The prompt for the Verifier. It sees the full state of the problem.
    prompt_verify = f"""Task: Evaluate if the current memory is complete and accurate enough to answer the query, or if more tools are needed.

Context:
- **Query:** {question}
- **Available Tools:** {json.dumps(self.available_tools)}
- **Toolbox Metadata:** {json.dumps(self.toolbox_metadata, indent=2)}
- **Initial Analysis:** {query_analysis}
- **Memory (Tools Used & Results):** {json.dumps(self.memory.get_actions())}

Instructions:
1.  Review the query, initial analysis, and memory.
2.  Assess the completeness of the memory: Does it fully address all parts of the query?
3.  Determine if any unused tools could provide missing information.
4.  If the memory is sufficient, explain why and set 'stop_signal' to true.
5.  If more information is needed, explain what's missing, which tools could help, and set 'stop_signal' to false.
"""
    # The LLM must respond in the format of the MemoryVerification Pydantic model.
    verification = self.llm_verifier.generate(prompt_verify, response_format=MemoryVerification)
    conclusion = 'STOP' if verification.stop_signal else 'CONTINUE'
    print(f"[Verifier Analysis]: {verification.analysis}\n[Verifier Conclusion]: {conclusion}")
    return verification.stop_signal

Solver.solve_step_4_verify = solve_step_4_verify

#### Final Step: Synthesizing the Answer
Once the Verifier signals to stop, the loop terminates. The agent now has all the necessary information in its memory. The final step is to use one last LLM call to synthesize all the actions and results into a single, coherent, human-readable answer that directly addresses the original user query.

In [26]:
def solve_final_step_synthesize(self, question: str, query_analysis: QueryAnalysis) -> str:
    """Generates the final, synthesized answer for the user."""
    print(f"\n{'='*80}\n==> 🐙 Generating Final Output (using planner_fixed_engine)\n{'='*80}")
    # The prompt for the final synthesis. It gets the query and the complete action history.
    prompt_final_output = f"""Task: Generate a concise final answer to the query based on all provided context.

Context:
- **Query:** {question}
- **Initial Analysis:** {query_analysis}
- **Actions Taken:** {json.dumps(self.memory.get_actions())}

Instructions:
1. Review the query and the results from all actions.
2. Synthesize the key findings into a clear, step-by-step summary of the process.
3. Provide a direct, precise answer to the original query.

Output Structure:
1.  **Process Summary:** A clear, step-by-step breakdown of how the query was addressed.
2.  **Answer:** A direct and concise final answer to the query."""
    # This is a free-form generation, no Pydantic model is needed.
    return self.llm_planner_fixed.generate(prompt_final_output)

Solver.solve_final_step_synthesize = solve_final_step_synthesize

#### The Main `solve` Method

Finally, we assemble all the previously defined step methods into the main `solve` method. This method orchestrates the calls to each step function in the correct order, manages the loop, and returns the final answer.

In [27]:
def solve(self, question: str):
    """Main method to run the entire agentic loop from start to finish."""
    # Step 0: Initial Analysis
    query_analysis = self.solve_step_0_analyze(question)
    
    step_count = 0
    while step_count < self.max_steps:
        step_count += 1
        
        # Step 1: Plan next action
        next_step = self.solve_step_1_plan(question, query_analysis, step_count)
        
        # Step 2: Generate command
        command_to_run = self.solve_step_2_generate_command(question, next_step, step_count)
        
        # Step 3: Execute command
        self.solve_step_3_execute_command(next_step, command_to_run, step_count)
        
        # Step 4: Verify and decide whether to continue
        should_stop = self.solve_step_4_verify(question, query_analysis, step_count)
        if should_stop:
            break
            
    # Final Step: Synthesize the final answer
    final_answer = self.solve_final_step_synthesize(question, query_analysis)
    return final_answer

# Monkey-patch the main solve method onto the Solver class.
Solver.solve = solve

## Section 7: Running the Agent

With all the components built and assembled, it's time to run our agent. The `construct_solver` function brings together the configuration for our specific run—defining which LLM model to use for each role and which tools should be enabled. Then, we provide a query and let the agent work.

In [28]:
def construct_solver():
    """Configures and constructs an instance of the Solver agent."""
    # Define which LLM model to use for each agent role.
    planner_main_engine, planner_fixed_engine, verifier_engine, executor_engine = MODEL_NAME, MODEL_NAME, MODEL_NAME, MODEL_NAME
    
    # Define the list of tools the agent should have access to.
    enabled_tools = ["Base_Generator_Tool", "Python_Coder_Tool", "Google_Search_Tool", "Wikipedia_RAG_Search_Tool"]
    
    # Define the specific LLM engine for each tool. 'Default' means the tool doesn't need an LLM.
    tool_engine = [MODEL_NAME, MODEL_NAME, "Default", MODEL_NAME]
    
    return Solver(planner_main_engine, planner_fixed_engine, verifier_engine, executor_engine, enabled_tools, tool_engine, max_steps=5)

# ===================
# RUN THE SOLVER
# ===================

# 1. Create the solver instance based on our configuration.
solver = construct_solver()

# 2. Define the complex, multi-step query we want the agent to solve.
query_to_solve = "Calculate 12 squared, and then use Wikipedia to find out what major historical event happened in that year (AD)."

# 3. Call the .solve() method to start the agentic workflow.
final_answer = solver.solve(query_to_solve)

# 4. Print the final, synthesized answer produced by the agent.
print(f"\n{'='*80}\n✅ FINAL SYNTHESIZED ANSWER\n{'='*80}\n{final_answer}")


==> Initializing LLM engines for different roles...
  - Planner (Main):      qwen-1.5-finetuned-grpo-fast
  - Planner (Fixed/Aux): qwen-1.5-finetuned-grpo-fast
  - Verifier:            qwen-1.5-finetuned-grpo-fast
  - Executor:            qwen-1.5-finetuned-grpo-fast

==> Initializing agent...
Enabled tools: ['Base_Generator_Tool', 'Python_Coder_Tool', 'Google_Search_Tool', 'Wikipedia_RAG_Search_Tool'] with engines: ['qwen-1.5-finetuned-grpo-fast', 'qwen-1.5-finetuned-grpo-fast', 'Default', 'qwen-1.5-finetuned-grpo-fast']
  -> Loading 'Base_Generator_Tool' with engine 'qwen-1.5-finetuned-grpo-fast'...
     ✓ Loaded and cached as 'Generalist_Solution_Generator_Tool'
  -> Loading 'Python_Coder_Tool' with engine 'qwen-1.5-finetuned-grpo-fast'...
     ✓ Loaded and cached as 'Python_Code_Generator_Tool'
  -> Loading 'Google_Search_Tool' with engine 'Default'...
     ✓ Loaded and cached as 'Ground_Google_Search_Tool'
  -> Loading 'Wikipedia_RAG_Search_Tool' with engine 'qwen-1.5-finetuned-g